In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

class BetaGo:
    def __init__(self, shape):
        #定义网络结构
        self.vars = []
        self.x = tf.placeholder(dtype=tf.float32, shape=[None, shape[0]])
        self.y = tf.placeholder(dtype=tf.float32, shape=[None, shape[-1]])
        pre_layer = self.x
        for index in range(1, len(shape) - 1):
            m = shape[index - 1]
            n = shape[index]
            w = tf.Variable(dtype=tf.float32, initial_value=tf.truncated_normal(shape=[m, n]))
            b = tf.Variable(dtype=tf.float32, initial_value=tf.truncated_normal(shape=[n]))
            self.vars.append(w)
            self.vars.append(b)
            pre_layer = tf.nn.relu(tf.matmul(pre_layer, w) + b)
        w = tf.Variable(dtype=tf.float32, initial_value=tf.truncated_normal(shape=shape[-2:]))
        b = tf.Variable(dtype=tf.float32, initial_value=tf.truncated_normal(shape=[shape[-1]]))
        self.vars.append(w)
        self.vars.append(b)
        self.y_predict = tf.matmul(pre_layer, w) +b
        
        
    def train(self, x_data, y_data):
        res = []
        self.obj = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.square(self.y_predict-self.y), 1)))
        tf.summary.scalar("loss", self.obj)
        merged = tf.summary.merge_all()
        
        self.step = tf.train.GradientDescentOptimizer(0.02).minimize(self.obj)
        self.init = tf.variables_initializer(var_list=self.vars)
        self.sess = tf.Session()
        self.writer = tf.summary.FileWriter("logs2", tf.get_default_graph())
        self.sess.run(self.init)
        for i in range(1, 3000):
            self.sess.run(self.step, feed_dict={self.x: x_data, self.y: y_data})
            dt = self.sess.run(merged, feed_dict={self.x: x_data, self.y: y_data})
            self.writer.add_summary(dt, i)
            if i % 20 == 0:
                res.append(self.sess.run(self.obj, feed_dict={self.x: x_data, self.y: y_data}))
        return res
    
    def batch_data(self, data):
        pass
    
    def predict(self, x_data):
        res = self.sess.run(self.y_predict, feed_dict={self.x: x_data})
        return res
        
x_data = np.random.sample([20, 12])
y_data = np.random.sample([20, 1])*300 - 150
model = BetaGo([12, 15, 1])


data = model.train(x_data, y_data)
#plt.plot(data)

y_predict = model.predict(x_data)